<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/lecture-idea/60_linear_algebra_2/150_Inverse_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
# This cell is for the Google Colaboratory
# https://stackoverflow.com/a/63519730
if 'google.colab' in str(get_ipython()):
  path_py = '/content/nmisp_py'

  import os
  if not os.path.exists(path_py):
    import subprocess
    subprocess.run(
        ('git', 'clone', 'https://github.com/kangwonlee/nmisp_py')
    )
  assert os.path.exists(path_py)

  import sys
  sys.path.insert(0, path_py)



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as nl
import numpy.random as nr
import sympy as sym
import IPython.display as disp

sym.init_printing()



The following function would visualize a numerical matrix using the [Hinton Diagram](https://matplotlib.org/stable/gallery/specialty_plots/hinton_demo.html).<br>
아래 함수는 행렬을 [힌튼 다이어그램](https://matplotlib.org/stable/gallery/specialty_plots/hinton_demo.html)으로 시각화할 것이다.


In [ ]:
def hinton(matrix, max_weight=None, ax=None, fname=None):
    '''
    Draw Hinton diagram for visualizing a weight matrix.
    https://matplotlib.org/stable/gallery/specialty_plots/hinton_demo.html
    '''
    ax = ax if ax is not None else plt.gca()

    if not max_weight:
        max_weight = 2 ** np.ceil(np.log2(np.abs(matrix).max()))

    ax.patch.set_facecolor('gray')
    ax.set_aspect('equal', 'box')

    ax.xaxis.set_major_locator(plt.NullLocator())
    ax.yaxis.set_major_locator(plt.NullLocator())

    for (y, x), w in np.ndenumerate(matrix):
        color = 'white' if w > 0 else 'black'
        size = np.sqrt(abs(w) / max_weight)
        rect = plt.Rectangle([x - size / 2, y - size / 2], size, size,
                             facecolor=color, edgecolor=color)
        ax.add_patch(rect)

    ax.autoscale_view()
    ax.invert_yaxis()

    if fname is None:
        plt.show()
        plt.close()
    else:
        plt.savefig(fname, dpi=300)
        plt.close()



In [ ]:
a, b, c, d = sym.symbols('a b c d')



# 역행렬<br>Inverse matrix



## 2x2



다음과 같은 2x2 행렬의 역행렬을 구하는 법은 이미 알고 있을 것이다.<br>
We know how we can find a 2x2 matrix as follows.



$$
A = \begin{bmatrix}
    a & b\\
    c & d \\
\end{bmatrix}
$$



In [ ]:
A = sym.Matrix([
    [a, b],
    [c, d]]
)
A



$$
A^{-1} = \frac {1}{ad-bc}\begin{bmatrix}
    d & -b\\
   -c & a \\
\end{bmatrix}
$$



여기서 $ad-bc$ 는 해당 행렬의 *행렬식* 이다.<br>
Here $ad-bc$ is the *determinant* of the matrix.



In [ ]:
def det22(matA):
    assert (2, 2) == matA.shape
    return matA[0, 0] * matA[1, 1] - matA[0, 1] * matA[1, 0]



그런데 행렬 부분은 어떠한가?<br>
What about the matrix part?


In [ ]:
adjA = sym.Matrix([
    [d, -b],
    [-c, a]]
)
adjA



원래의 행렬과 곱해보자.<br>
Let's multiply with the original matrix.



In [ ]:
A * adjA



결과는 어떠한 형태인가?<br>
What does the result look like?



임의의 크기의 행렬에 대해 그러한 행렬을 찾을 수 있을까?<br>
Can we find such matrix for a matrix of an arbitrary size?



## 행렬식<br>Determinant



### 3x3




다음과 같은 3x3 행렬을 생각해 보자.<br>
Let's think about a 3x3 matrix as follows.



In [ ]:
e, f, g, h, i = sym.symbols(list('efghi'))



In [ ]:
A = sym.Matrix([
    [a, b, c],
    [d, e, f,],
    [g, h, i,],
])
A



해당 행렬의 행렬식을 구해보자.<br>
Let's find the determinant of the matrix.



#### 소행렬식<br>Minor




$i$ $j$ 번째 행을 삭제한 행렬을 생각해 보자.<br>
Let's consider a matrix without $i$-th row and $j$-th column.



In [ ]:
def row_col_del(matA, row_i, col_j):
    result = sym.Matrix(matA)
    result.row_del(row_i)
    result.col_del(col_j)
    return result



In [ ]:
A00 = row_col_del(A, 0, 0)
A00



In [ ]:
A01 = row_col_del(A, 0, 1)
A01



In [ ]:
A02 = row_col_del(A, 0, 2)
A02



행렬 $A_{ij}$ 는 $A$ 행렬에서 $i$행 $j$열을 제외한 $\left(n-1\right) \times \left(n-1\right)$ 행렬이다 .<br>
Matrix $A_{ij}$ is a $\left(n-1\right) \times \left(n-1\right)$ matrix excluding $i$-th row and $j$-th column of the matrix $A$.



행렬식은 다음과 같이 계산할 수 있다.  이를 *소행렬식* 이라 한다.<br>
We can calculate the determinant as follows.  This is called the *Minor*.



In [ ]:
det1 = A[0, 0] * det22(A00) - A[0, 1] * det22(A01) + A[0, 2] * det22(A02)
det1



함수 형태로는 다음과 같다.<br>
We may write in a function as follows.



In [ ]:
def det33(matA:np.ndarray) -> float:

    result = 0

    for j in range(matA.shape[1]):
        result += ((-1) ** j) * matA[0, j] * det22(row_col_del(matA, 0, j))

    return result



리스트 줄여쓰기로 다음과 같이 구현할 수도 있다.<br>
We can implement as follows using the list comprehension.



In [ ]:
def det33_list_comprehension(matA:np.ndarray) -> float:
    return sum(
        [((-1) ** j) * matA[0, j] * det22(row_col_del(matA, 0, j)) for j in range(matA.shape[1])]
    )



아래 셀은 함수를 확인한다.<br>
Following cell checks the function.



In [ ]:
assert 0 == sym.simplify(det33(A) - det1)
assert 0 == sym.simplify(det33_list_comprehension(A) - det1)



### n x n, n > 3



일반화 하면 다음과 같다.<br>
We can generalize as follows.



In [ ]:
def det(matA:np.ndarray) -> float:
    if 2 == matA.shape[0]:
        result = det22(matA)
    elif 2 < matA.shape[0]:
        result = sum(
            [((-1) ** j) * matA[0, j] * det(row_col_del(matA, 0, j)) for j in range(matA.shape[1])]
        )
    return result



## 여인수행렬<br>Cofactor matrix



행렬 $A_{ij}$ 의 행렬식, *소행렬식* 에 번갈아 부호를 붙인 것을 *여인수* $C_{ij}$ 라고 한다.<br>
The signed version of the *minor*, determinant of the matrix $A_{ij}$, is the *cofactor* $C_{ij}$.



In [ ]:
def cofactor(A, i, j):
    return det(row_col_del(A, i, j))



여인수로 이루어진 행렬을 생각해 보자.<br>
Let's think about the matrix of cofactors.



In [ ]:
def cofactor_matrix(matA):
       return sym.Matrix([[cofactor(matA, i, j) * ((-1) ** (i+j)) for j in range(matA.shape[1])] for i in range(matA.shape[0])])



In [ ]:
cofactor_A = cofactor_matrix(A)
cofactor_A



*여인수행렬* 의 전치행렬을 *수반행렬* 이라고 한다.<br>
The transpose of a *cofactor matrix* is the *adjugate matrix*.



원래 행렬 $A$와 *수반행렬* 을 곱해 보자.<br>
Let's multiply the origitnal matrix $A$ and its *adjugate matrix*.



In [ ]:
sym.simplify(A * cofactor_A.T)



위 행렬을 $A$의 행렬식으로 나누어 보자.<br>
Let's divide the matrix above with the determinant of $A$.



In [ ]:
sym.simplify(A * cofactor_A.T / det1)



위 결과는 어떠한가?<br>
How is the result above?



## 2x2 : Gauss Jordan Method



다음 비디오는 역행열 찾는 가우스 조단법을 소개한다.<br>
Following video introduces Gauss Jordan method finding the inverse matrix. (36:23 ~ 42:20)

[![MIT OCW 18.06 Lecture 3 Multiplication and Inverse Matrices](https://i.ytimg.com/vi/FX4C-JpTFgY/hqdefault.jpg)](https://www.youtube.com/watch?v=FX4C-JpTFgY&list=PL221E2BBF13BECF6C&index=9&start=2183&end=2540)


아래 2x2 행렬을 생각해 보자.<br>
Let's think about the 2x2 matrix.



In [ ]:
A22 = np.array([
    [1, 3],
    [2, 7]
])



In [ ]:
A22



In [ ]:
hinton(A22)



오른쪽에 같은 크기의 단위행렬을 붙여 보자.<br>
Let's augment an identity matrix of the same size.



In [ ]:
I22 = np.identity(2)



In [ ]:
I22



In [ ]:
AX22 = np.hstack([A22, I22])



In [ ]:
AX22



In [ ]:
hinton(AX22)



이제 왼쪽 2x2 부분을 단위행렬로 만들어 보자.<br>
Let's make the left 2x2 part an identity matrix.



첫 행에 2를 곱한 후 2행에서 빼 보자.<br>
Let's multipy 2 to the first row and then subtract from the second row.



In [ ]:
AX22[1, :] -= 2 * AX22[0, :]



In [ ]:
AX22



In [ ]:
hinton(AX22)



이번에는 2번째 행에 3을 곱해서 첫 행에서 빼 보자.<br>
Now let's multipy 3 to the second row and subtract from the first row.



In [ ]:
AX22[0, :] -= 3 * AX22[1, :]



In [ ]:
AX22



In [ ]:
hinton(AX22)



위 `AX22` 행렬에서 오른쪽 두 행을 따로 떼어 보자.<br>
Let's separate the right two columns of the `AX22` matrix above.



In [ ]:
A22_inv = AX22[:, 2:]



In [ ]:
A22_inv



In [ ]:
hinton(A22_inv)



`A22` 행렬과 곱해보자.<br>
Let's multipy with the A matrix.



In [ ]:
A22 @ A22_inv



## `numpy`



`numpy.linalg.inv()` function is available.<br>
`numpy.linalg.inv()` 함수로 구할 수 있다.



In [ ]:
A22_inv = nl.inv(A22)



In [ ]:
A22_inv



In [ ]:
A22_inv @ A22



## 3x3



다음 비디오는 3x3 행렬에 가우스 조단법을 적용한다.<br>
Following video applies the Gauss Jordan method to invert a 3x3 matrix.

[![Khan Academy inverting 3x3 matrix part 2](https://i.ytimg.com/vi/obts_JDS6_Q/hqdefault.jpg)](https://www.youtube.com/watch?v=obts_JDS6_Q)


아래 행렬을 생각해 보자.<br>
Let's think about the following matrix.



In [ ]:
A33_list = [
    [1, 0, 1],
    [0, 2, 1],
    [1, 1, 1],
]



In [ ]:
A33_list



In [ ]:
hinton(A33_list)



In [ ]:
A33 = np.array(A33_list)



In [ ]:
A33



In [ ]:
hinton(A33)



마찬가지로, 오른쪽에 같은 크기의 단위행렬을 붙여 보자.<br>
Same as before, let's augment an identity matrix of the same size.



In [ ]:
I33 = np.identity(A33.shape[0])



In [ ]:
I33



In [ ]:
AX33 = np.hstack([A33, I33])



In [ ]:
AX33



In [ ]:
hinton(AX33)



이제 왼쪽 부분을 단위행렬로 만들어 보자.<br>
Let's make the left part an identity matrix.



첫 행을 3행에서 빼 보자.<br>
Let's subtract the first row from the third row.



In [ ]:
AX33[2, :] -= AX33[0, :]



In [ ]:
AX33



In [ ]:
hinton(AX33)



이번에는 2번째 행과 3번째 행을 바꾸자.<br>
Now let's swap the second and the third rows. ([ref](https://stackoverflow.com/a/54069951))



In [ ]:
AX33[[1, 2]] = AX33[[2, 1]]



In [ ]:
AX33



In [ ]:
hinton(AX33)



두번째 행에 2를 곱해서 3행에서 빼 보자.<br>
Let's multiply 2 to the second row and subtract from the third row.



In [ ]:
AX33[2, :] -= 2 * AX33[1, :]



In [ ]:
AX33



In [ ]:
hinton(AX33)



첫번째 행에서 3번째 행을 빼 보자.<br>
Let's subtract the third row from the first row.



In [ ]:
AX33[0, :] -= AX33[2, :]



In [ ]:
AX33



In [ ]:
hinton(AX33)



위 `AX` 행렬에서 오른쪽 세 열을 따로 떼어 보자.<br>
Let's separate the right three columns of the `AX` matrix above.



In [ ]:
A33_inv = AX33[:, 3:]



In [ ]:
A33_inv



In [ ]:
hinton(A33_inv)



`A33` 행렬과 곱해보자.<br>
Let's multipy with the `A33` matrix.



In [ ]:
A33 @ A33_inv



## `numpy`



Please use `numpy.linlag.inv()`<br>`numpy.linlag.inv()`를 사용하자.



In [ ]:
mat_A33_inv = nl.inv(A33)



In [ ]:
mat_A33_inv



In [ ]:
A33 @ mat_A33_inv



## 표준 기능으로 구현한 가우스 조단법<br>Gauss Jordan method in Standard library



다음 셀은 가우스 조단법을 표준기능 만으로 구현한다.<br>
Following cell implements the Gauss Jordan method with standard library only.



In [ ]:
import typing

Scalar = typing.Union[int, float]
Row = typing.Union[typing.List[Scalar], typing.Tuple[Scalar]]
Matrix = typing.Union[typing.List[Row], typing.Tuple[Row]]


def get_zero(n:int) -> Matrix:
    return [
        [0] * n for i in range(n)
    ]


def get_identity(n:int) -> Matrix:
    result = get_zero(n)
    for i in range(n):
        result[i][i] = 1

    return result


def augment_mats(A:Matrix, B:Matrix):
    assert len(A) == len(B)
    return [row_A + row_B for row_A, row_B in zip(A, B)]


def gauss_jordan(A:Matrix, b_hinton:bool=False, epsilon=1e-7, b_savefig=False) -> Matrix:
    AX = augment_mats(A, get_identity(len(A)))

    counter = 0

    # pivot loop
    for p in range(len(AX)):

        assert abs(AX[p][p]) > epsilon, (p, AX)
        one_over_pivot = 1.0 / AX[p][p]

        # normalize a row with one_over_pivot
        for j in range(len(AX[p])):
            AX[p][j] *= one_over_pivot

        # row loop
        for i in range(len(AX)):
            if i != p:
                # row operation
                multiplier = - AX[i][p]

                # column loop
                for j in range(0, len(AX[p])):
                    AX[i][j] += multiplier * AX[p][j]

            # visualize augmented matrix 
            if b_hinton:
                if b_savefig:
                    hinton(AX, fname=f'GJ{len(A)}{len(A)}-{counter:04d}.png')
                    counter += 1
                else:
                    hinton(AX)

    return [row[len(A):] for row in AX]



위 행렬의 예로 확인해 보자.<br>
Let's check with the matrix above.



In [ ]:
mat_A33_inv_GJ = gauss_jordan(A33_list, b_hinton=True, b_savefig=False)



In [ ]:
import pprint
pprint.pprint(mat_A33_inv_GJ, width=40)



In [ ]:
np.array(mat_A33_inv_GJ) @ A33



## 4x4



아래 행렬을 생각해 보자.<br>
Let's think about the following matrix.



In [ ]:
A44_list = [
    [1, 0, 2, 0],
    [1, 1, 0, 0],
    [1, 2, 0, 1],
    [1, 1, 1, 1],
]



In [ ]:
A44 = np.array(A44_list, dtype=float)



In [ ]:
A44



In [ ]:
hinton(A44)



다음 셀은 넘파이 다차원 배열 `numpy.ndarray` 을 위해 구현한 가우스 조르단 소거법 함수를 불러들인다.<br>
Following cell imports an implementation of the Gauss Jordan Elimination for a `numpy.ndarray`.



In [ ]:
import gauss_jordan as gj



위 행렬에 적용해 보자.<br>
Let's apply to the matrix above.



In [ ]:
A44_inv_array = gauss_jordan(A44.tolist(), b_hinton=True, b_savefig=False)



In [ ]:
A44_inv_array



In [ ]:
A44_inv_array @ A44



In [ ]:
import numpy.testing as nt


nt.assert_allclose(A44_inv_array @ A44, np.array(get_identity(len(A44_list))))



### n x n



좀 더 큰 행렬의 경우, 여인수행렬과 비교해 보자.<br>
Let's compare with the cofactor matrix.



In [ ]:
n = 7
Ann = nr.random((n, n))



In [ ]:
hinton(Ann)



In [ ]:
inv1 = gauss_jordan(Ann.tolist(), b_hinton=True, b_savefig=False)



In [ ]:
%time inv1 = gauss_jordan(Ann.tolist())



Let's compare the computation time with the cofactor matrix algorithm.<br>
여인수 행렬 알고리듬과 계산 시간을 비교해 보자.



In [ ]:
%time inv2 = (1/det(Ann)) * cofactor_matrix(Ann).T
inv2_array = np.array(inv2.tolist(), dtype=float)



In [ ]:
import numpy.testing as nt
nt.assert_allclose(inv1, inv2_array)



## 연습 문제<br>Exercise



위 가우스 조단법에서 메모리를 더 절약하는 방안을 제안해 보시오<br>
Regarding the Gauss Jordan implementation above, propose how we can save more memory.



## 소거행렬<br>Elimination Matrix




### 2x2



다시 한번 단위 행렬을 덧붙인 행렬을 생각해 보자.<br>
Let's think about the augmented matrix again.



In [ ]:
AX22 = np.hstack([A22, I22])
AX22



(1, 0) 위치의 요소를 0으로 만드는 소거 행렬을 생각해 보자.<br>
Let's think about the elimination matrix making element at (1, 0) location.



In [ ]:
E10 = np.identity(A22.shape[0])
E10[1, 0] = -AX22[1, 0] / AX22[0, 0]
E10



In [ ]:
AX22 = E10 @ AX22
AX22



이번에는, (0, 1) 위치의 요소를 0으로 만드는 소거 행렬을 생각해 보자.<br>
Let's think about the elimination matrix making element at (0, 1) location.



In [ ]:
E01 = np.identity(A22.shape[0])
E01[0, 1] = -AX22[0, 1] / AX22[1, 1]
E01



In [ ]:
AX22 = E01 @ AX22
AX22



지금까지의 소거행렬을 순서 대로 곱해보자.<br>
Let's multipy the elimination matrices in the order.



In [ ]:
E01 @ E10



위에서 구했던 역행렬도 다음과 같다.<br>
The inverse matrix we found above were as follows, too.



In [ ]:
A22_inv



## 참고문헌<br>References



* Gilbert Strang. 18.06 Linear Algebra. Spring 2010. Massachusetts Institute of Technology: MIT OpenCourseWare, https://ocw.mit.edu. License: Creative Commons BY-NC-SA.
* Marc Peter Deisenroth, A Aldo Faisal, and Cheng Soon Ong, Mathematics For Machine Learning, Cambridge University Press, 2020, ISBN 978-1108455145.
* Erwin Kreyszig, Advanced Engineering Mathematics, 5th Ed, John Wiley & Sons, 1983, ISBN 0-471-86251-7.



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

